# Student Loan Risk with Deep Learning

In [1]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [2]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,1
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,0
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,1
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,1
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,0


In [3]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [4]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [7]:
# Define the target set y using the credit_ranking column

y = loans_df["credit_ranking"]

# Display a sample of y
y

0       0
1       0
2       0
3       1
4       0
       ..
1594    0
1595    1
1596    1
1597    0
1598    1
Name: credit_ranking, Length: 1599, dtype: int64

In [8]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1)

# Review the features DataFrame
X

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


### Step 3: Split the features and target sets into training and testing datasets.


In [9]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [10]:
# Create a StandardScaler instance
X_scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [12]:
# Define the the number of inputs (features) to the model
nn_model = tf.keras.models.Sequential()

# Review the number of features
input_nodes = len(X.columns)

input_nodes

11

In [13]:
# Define the number of hidden nodes for the first hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu", input_dim=input_nodes))

# Define the number of hidden nodes for the second hidden layer
nn_model.add(tf.keras.layers.Dense(units=5, activation="relu"))

# Define the number of neurons in the output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

In [ ]:
# Create the Sequential model instance
## Completed above

# Add the first hidden layer
## Completed above

# Add the second hidden layer
## Completed above

# Add the output layer to the model specifying the number of output neurons and activation function
## Completed above

In [14]:
# Display the Sequential model summary
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 60        
                                                                 
 dense_1 (Dense)             (None, 5)                 30        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 96 (384.00 Byte)
Trainable params: 96 (384.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [15]:
# Compile the Sequential model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [16]:
# Fit the model using 50 epochs and the training data
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
38/38 [==============================] - 1s 3ms/step - loss: 0.6824 - accuracy: 0.5455
Epoch 2/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6638 - accuracy: 0.6005
Epoch 3/50
38/38 [==============================] - 0s 4ms/step - loss: 0.6507 - accuracy: 0.6597
Epoch 4/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6376 - accuracy: 0.6922
Epoch 5/50
38/38 [==============================] - 0s 3ms/step - loss: 0.6228 - accuracy: 0.7064
Epoch 6/50
38/38 [==============================] - 0s 2ms/step - loss: 0.6081 - accuracy: 0.7131
Epoch 7/50
38/38 [==============================] - 0s 2ms/step - loss: 0.5947 - accuracy: 0.7181
Epoch 8/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5805 - accuracy: 0.7306
Epoch 9/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5675 - accuracy: 0.7339
Epoch 10/50
38/38 [==============================] - 0s 3ms/step - loss: 0.5565 - accuracy: 0.7364
Epoch 11/50
38/38 [

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [17]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - loss: 0.5056 - accuracy: 0.7525 - 277ms/epoch - 21ms/step
Loss: 0.5055541396141052, Accuracy: 0.7524999976158142


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [18]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Export your model to a keras file
nn_model.save(file_path)

---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [19]:
# Set the model's file path
file_path = Path("student_loans.keras")

# Load the model to a new object
nn_imported = tf.keras.models.load_model(file_path)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [20]:
# Make predictions with the test data
predictions = nn_model.predict(X_test_scaled,verbose=2)


# Display a sample of the predictions
predictions

13/13 - 0s - 106ms/epoch - 8ms/step


array([[0.29971588],
       [0.43419787],
       [0.82024354],
       [0.7375767 ],
       [0.96688706],
       [0.80511826],
       [0.8981182 ],
       [0.04459045],
       [0.5856588 ],
       [0.33905518],
       [0.8964389 ],
       [0.2606675 ],
       [0.4633378 ],
       [0.9018801 ],
       [0.72251177],
       [0.41960198],
       [0.92079026],
       [0.32611236],
       [0.58174616],
       [0.41249636],
       [0.6884224 ],
       [0.8310817 ],
       [0.28622445],
       [0.77506125],
       [0.30965224],
       [0.941143  ],
       [0.7581384 ],
       [0.7828294 ],
       [0.25256306],
       [0.29877126],
       [0.7228768 ],
       [0.8895021 ],
       [0.24948986],
       [0.89078146],
       [0.16027865],
       [0.51624215],
       [0.2471118 ],
       [0.36317465],
       [0.9229961 ],
       [0.20442802],
       [0.9061726 ],
       [0.04386217],
       [0.16195574],
       [0.8960748 ],
       [0.050931  ],
       [0.71250784],
       [0.21006593],
       [0.297

In [21]:
# Save the predictions to a DataFrame and round the predictions to binary results
predictions_df = pd.DataFrame(columns=["predictions"], data=predictions)
predictions_df["predictions"] = round(predictions_df["predictions"],0)
predictions_df

,predictions
0,0.0
1,0.0
2,1.0
3,1.0
4,1.0
...,...
395,1.0
396,0.0
397,1.0
398,0.0


### Step 4: Display a classification report with the y test data and predictions

In [22]:
# Print the classification report with the y test data and predictions
print(classification_report(y_test, predictions_df["predictions"].values))

              precision    recall  f1-score   support

           0       0.72      0.78      0.75       188
           1       0.79      0.73      0.76       212

    accuracy                           0.75       400
   macro avg       0.75      0.75      0.75       400
weighted avg       0.76      0.75      0.75       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
It would be helpful to have data on the details of each student's job after graduation. For example, if it is a part-time job or has low salary, you would want to recommend an option for the student to pay the loan off in smaller amounts over a longer time period with a higher interest rate. It would also be beneficial to have a summary of their current liquid assets to recommend options for students to pay off the loans all at once if they want. It would also be helpful if the models could flag which students should not pay off the loans all at once and educate them on why it may be in their interest to save their money now and pay the loan off over time if the model flags that the student may not be in a strong financial position to pay off the loans immediately.


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**
Context based filtering because the student's job salary details and liquid assets would be contextual for each individual student.


**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
Providing personal asset information such as liquid assets is a privacy concern for students / recent graduates. The other concern is what I mentioned above about educating students on the pros & cons of each choice in the recommendation model on student loan options. Students may not fully understand the long-term financial implications of their decisions. There is also a conflict of interest in that the company providing the refinancing options / recommendation model also want the most amount of money from each student without the student defaulting on their loans, so the recommendation on the specific loan options may not be in the student's best interest vs other options the refinancing company offers.


